In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from typing import List, Tuple, Optional

class BatchNormalizationLayer(nn.Module):
    """
    Custom Batch Normalization Layer as described in the paper.
    Implements both training and inference modes with moving averages.
    """
    def __init__(self, input_size: int, momentum: float = 0.1, eps: float = 1e-5):
        super(BatchNormalizationLayer, self).__init__()
        self.input_size = input_size
        self.momentum = momentum  # κ in the paper
        self.eps = eps  # ϵ in the paper
        
        # Learnable parameters γ and β
        self.gamma = nn.Parameter(torch.ones(input_size))
        self.beta = nn.Parameter(torch.zeros(input_size))
        
        # Moving averages for inference
        self.register_buffer('running_mean', torch.zeros(input_size))
        self.register_buffer('running_var', torch.ones(input_size))
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.training:
            # Training phase: use batch statistics
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0, unbiased=False)
            
            # Update moving averages
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * batch_mean
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * batch_var
            
            # Normalize using batch statistics
            x_normalized = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
        else:
            # Inference phase: use moving averages
            x_normalized = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)
        
        # Apply scale and shift
        return self.gamma * x_normalized + self.beta


class MultiFaultDiagnosisNN(nn.Module):
    """
    Multi-fault diagnosis neural network as described in Fig. 2(a) of the paper.
    Uses binary cross-entropy loss for multi-label classification.
    """
    def __init__(self, input_size: int, num_faults: int, hidden_layers: List[int] = [128, 64, 32]):
        super(MultiFaultDiagnosisNN, self).__init__()
        self.input_size = input_size
        self.num_faults = num_faults
        
        # Batch normalization as first layer
        self.batch_norm = BatchNormalizationLayer(input_size)
        
        # Build fully connected layers
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        
        # Output layer with sigmoid activation
        layers.append(nn.Linear(prev_size, num_faults))
        
        self.fc_layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Apply batch normalization first
        x = self.batch_norm(x)
        
        # Pass through FC layers
        x = self.fc_layers(x)
        
        # Apply sigmoid activation for multi-label classification
        return torch.sigmoid(x)
    
    def predict(self, x: torch.Tensor, threshold: float = 0.5) -> torch.Tensor:
        """Online inference with threshold decision"""
        self.eval()
        with torch.no_grad():
            output = self.forward(x)
            predictions = (output > threshold).float()
        return predictions


class SeverityDiagnosisNN(nn.Module):
    """
    Severity diagnosis neural network for individual fault types.
    Uses categorical cross-entropy for multi-class classification.
    """
    def __init__(self, input_size: int, num_severity_levels: int = 3, 
                 hidden_layers: List[int] = [128, 64, 32]):
        super(SeverityDiagnosisNN, self).__init__()
        self.input_size = input_size
        self.num_severity_levels = num_severity_levels
        
        # Batch normalization as first layer
        self.batch_norm = BatchNormalizationLayer(input_size)
        
        # Build fully connected layers
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        
        # Output layer
        layers.append(nn.Linear(prev_size, num_severity_levels))
        
        self.fc_layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Apply batch normalization first
        x = self.batch_norm(x)
        
        # Pass through FC layers
        x = self.fc_layers(x)
        
        # Apply softmax for multi-class classification
        return F.softmax(x, dim=1)
    
    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """Online inference with argmax decision"""
        self.eval()
        with torch.no_grad():
            output = self.forward(x)
            predictions = torch.argmax(output, dim=1)
        return predictions


class SeparatedStructure(nn.Module):
    """
    Separated structure for fault and severity diagnosis.
    Contains one fault diagnosis NN and NF severity diagnosis NNs.
    """
    def __init__(self, input_size: int, num_faults: int, num_severity_levels: int = 3,
                 hidden_layers: List[int] = [128, 64, 32]):
        super(SeparatedStructure, self).__init__()
        self.num_faults = num_faults
        self.num_severity_levels = num_severity_levels
        
        # Fault diagnosis network
        self.fault_diagnosis = MultiFaultDiagnosisNN(input_size, num_faults, hidden_layers)
        
        # Severity diagnosis networks (one for each fault type)
        self.severity_networks = nn.ModuleList([
            SeverityDiagnosisNN(input_size, num_severity_levels, hidden_layers)
            for _ in range(num_faults)
        ])
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, List[torch.Tensor]]:
        # Diagnose faults
        fault_predictions = self.fault_diagnosis(x)
        
        # Diagnose severity for each fault type
        severity_predictions = []
        for i, severity_net in enumerate(self.severity_networks):
            severity_pred = severity_net(x)
            severity_predictions.append(severity_pred)
        
        return fault_predictions, severity_predictions
    
    def predict(self, x: torch.Tensor, fault_threshold: float = 0.5) -> Tuple[torch.Tensor, List[torch.Tensor]]:
        """Combined prediction for faults and severities"""
        self.eval()
        with torch.no_grad():
            fault_pred = self.fault_diagnosis.predict(x, fault_threshold)
            severity_preds = []
            
            for severity_net in self.severity_networks:
                severity_pred = severity_net.predict(x)
                severity_preds.append(severity_pred)
        
        return fault_pred, severity_preds


class JointStructure(nn.Module):
    """
    Joint structure for simultaneous fault and severity diagnosis.
    Uses a single NN for both fault detection and severity classification.
    """
    def __init__(self, input_size: int, num_faults: int, num_severity_levels: int = 3,
                 hidden_layers: List[int] = [128, 64, 32]):
        super(JointStructure, self).__init__()
        self.input_size = input_size
        self.num_faults = num_faults
        self.num_severity_levels = num_severity_levels
        self.output_size = num_faults * num_severity_levels
        
        # Batch normalization as first layer
        self.batch_norm = BatchNormalizationLayer(input_size)
        
        # Build fully connected layers
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        
        # Output layer for joint fault-severity classification
        layers.append(nn.Linear(prev_size, self.output_size))
        
        self.fc_layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Apply batch normalization first
        x = self.batch_norm(x)
        
        # Pass through FC layers
        x = self.fc_layers(x)
        
        # Apply sigmoid activation for multi-label classification
        return torch.sigmoid(x)
    
    def predict(self, x: torch.Tensor, threshold: float = 0.5) -> torch.Tensor:
        """Online inference with threshold decision"""
        self.eval()
        with torch.no_grad():
            output = self.forward(x)
            predictions = (output > threshold).float()
        return predictions
    
    def decode_predictions(self, predictions: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Decode joint predictions into fault and severity predictions.
        Returns: (fault_predictions, severity_predictions)
        """
        batch_size = predictions.shape[0]
        fault_predictions = torch.zeros(batch_size, self.num_faults)
        severity_predictions = torch.zeros(batch_size, self.num_faults)
        
        for i in range(self.num_faults):
            start_idx = i * self.num_severity_levels
            end_idx = start_idx + self.num_severity_levels
            
            # Check if any severity level is predicted for this fault
            fault_severity_preds = predictions[:, start_idx:end_idx]
            fault_predictions[:, i] = torch.any(fault_severity_preds > 0.5, dim=1).float()
            
            # Get the highest severity level predicted
            severity_predictions[:, i] = torch.argmax(fault_severity_preds, dim=1).float()
        
        return fault_predictions, severity_predictions


class FaultDiagnosisTrainer:
    """
    Training utilities for fault diagnosis networks.
    """
    def __init__(self, model: nn.Module, learning_rate: float = 0.001):
        self.model = model
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
    def train_multi_fault(self, train_loader, num_epochs: int = 100):
        """Train multi-fault diagnosis network"""
        self.model.train()
        criterion = nn.BCELoss()
        
        for epoch in range(num_epochs):
            total_loss = 0.0
            for batch_idx, (data, targets) in enumerate(train_loader):
                self.optimizer.zero_grad()
                
                outputs = self.model(data)
                loss = criterion(outputs, targets.float())
                
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
            
            if (epoch + 1) % 10 == 0:
                avg_loss = total_loss / len(train_loader)
                print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')
    
    def train_severity(self, train_loader, num_epochs: int = 100):
        """Train severity diagnosis network"""
        self.model.train()
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(num_epochs):
            total_loss = 0.0
            for batch_idx, (data, targets) in enumerate(train_loader):
                self.optimizer.zero_grad()
                
                outputs = self.model(data)
                loss = criterion(outputs, targets.long())
                
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
            
            if (epoch + 1) % 10 == 0:
                avg_loss = total_loss / len(train_loader)
                print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')
    
    def train_joint(self, train_loader, num_epochs: int = 100):
        """Train joint fault-severity diagnosis network"""
        self.model.train()
        criterion = nn.BCELoss()
        
        for epoch in range(num_epochs):
            total_loss = 0.0
            for batch_idx, (data, targets) in enumerate(train_loader):
                self.optimizer.zero_grad()
                
                outputs = self.model(data)
                loss = criterion(outputs, targets.float())
                
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
            
            if (epoch + 1) % 10 == 0:
                avg_loss = total_loss / len(train_loader)
                print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')


# Example usage and demonstration
if __name__ == "__main__":
    # Example parameters
    input_size = 50  # Number of KPIs
    num_faults = 3   # ERP, ED, EU
    num_severity_levels = 3
    batch_size = 32
    
    # Generate sample data
    X = torch.randn(1000, input_size)
    y_faults = torch.randint(0, 2, (1000, num_faults))  # Multi-label fault data
    y_joint = torch.randint(0, 2, (1000, num_faults * num_severity_levels))  # Joint labels
    
    print("=== Multi-Fault Diagnosis Network ===")
    fault_model = MultiFaultDiagnosisNN(input_size, num_faults)
    print(fault_model)
    
    # Test forward pass
    with torch.no_grad():
        sample_input = torch.randn(1, input_size)
        fault_output = fault_model(sample_input)
        fault_prediction = fault_model.predict(sample_input)
        print(f"Fault probabilities: {fault_output}")
        print(f"Fault predictions: {fault_prediction}")
    
    print("\n=== Severity Diagnosis Network ===")
    severity_model = SeverityDiagnosisNN(input_size, num_severity_levels)
    print(severity_model)
    
    print("\n=== Separated Structure ===")
    separated_model = SeparatedStructure(input_size, num_faults, num_severity_levels)
    print(f"Number of parameters: {sum(p.numel() for p in separated_model.parameters())}")
    
    print("\n=== Joint Structure ===")
    joint_model = JointStructure(input_size, num_faults, num_severity_levels)
    print(f"Number of parameters: {sum(p.numel() for p in joint_model.parameters())}")
    
    # Test joint structure
    with torch.no_grad():
        joint_output = joint_model(sample_input)
        joint_prediction = joint_model.predict(sample_input)
        fault_pred, severity_pred = joint_model.decode_predictions(joint_prediction)
        print(f"Joint output shape: {joint_output.shape}")
        print(f"Decoded fault predictions: {fault_pred}")
        print(f"Decoded severity predictions: {severity_pred}")

=== Multi-Fault Diagnosis Network ===
MultiFaultDiagnosisNN(
  (batch_norm): BatchNormalizationLayer()
  (fc_layers): Sequential(
    (0): Linear(in_features=50, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=3, bias=True)
  )
)
Fault probabilities: tensor([[0.5310, 0.5158, 0.4802]])
Fault predictions: tensor([[1., 1., 0.]])

=== Severity Diagnosis Network ===
SeverityDiagnosisNN(
  (batch_norm): BatchNormalizationLayer()
  (fc_layers): Sequential(
    (0): Linear(in_features=50, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=3, bias=True)
  )
)

=== Separated Structure ===
Number of parameters: 68252

In [6]:
import numpy as np
import pandas as pd

def preprocess_kpis(df: pd.DataFrame, debug: bool = False) -> np.ndarray:
    """
    Preprocess network KPIs for SON fault diagnosis as per 2024 IEEE methodology.

    Parameters:
        df (pd.DataFrame): DataFrame with columns 'CQI', 'RSRP', 'SINR', 'TP'.
        debug (bool): If True, prints ECDF percentiles and normalization ranges.

    Returns:
        np.ndarray: Normalized 36-element vector (shape: (36,))
    """

    # 1. Input validation
    required_cols = ['CQI', 'RSRP', 'SINR', 'TP']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column: '{col}'")
    if df[required_cols].isnull().any().any():
        raise ValueError("Input DataFrame contains missing values in required columns.")

    # CQI: integer in [1, 15]
    if not np.issubdtype(df['CQI'].dtype, np.integer):
        if not np.all(df['CQI'] == df['CQI'].astype(int)):
            raise ValueError("CQI values must be integers in [1, 15].")
        df['CQI'] = df['CQI'].astype(int)
    if not ((df['CQI'] >= 1) & (df['CQI'] <= 15)).all():
        raise ValueError("CQI values must be integers in [1, 15].")

    # RSRP: float in [-144, -44]
    if not np.issubdtype(df['RSRP'].dtype, np.floating):
        df['RSRP'] = df['RSRP'].astype(float)
    if not ((df['RSRP'] >= -144) & (df['RSRP'] <= -44)).all():
        raise ValueError("RSRP values must be floats in [-144, -44].")

    # SINR: non-negative float
    if not np.issubdtype(df['SINR'].dtype, np.floating):
        df['SINR'] = df['SINR'].astype(float)
    if not (df['SINR'] >= 0).all():
        raise ValueError("SINR values must be non-negative floats.")

    # TP: non-negative float
    if not np.issubdtype(df['TP'].dtype, np.floating):
        df['TP'] = df['TP'].astype(float)
    if not (df['TP'] >= 0).all():
        raise ValueError("TP values must be non-negative floats.")

    # 2. ECDF and percentile extraction
    percentiles = np.arange(10, 100, 10)  # 10th, 20th, ..., 90th
    kpi_vectors = []
    normalization_info = {}

    for kpi, norm in zip(
        ['CQI', 'RSRP', 'SINR', 'TP'],
        ['cqi', 'rsrp', 'sinr', 'tp']
    ):
        values = df[kpi].values
        # Compute ECDF percentiles using numpy's percentile (linear interpolation)
        kpi_percentiles = np.percentile(values, percentiles, interpolation='linear')
        kpi_vectors.append(kpi_percentiles)
        normalization_info[norm] = {
            'raw_percentiles': kpi_percentiles.copy(),
            'min': values.min(),
            'max': values.max()
        }

    # 3. Concatenate into a single vector
    x = np.concatenate(kpi_vectors)  # shape (36,)

    # 4. Normalization
    # CQI: [1, 15] -> [0, 1]
    x[0:9] = x[0:9] / 15.0

    # RSRP: [-144, -44] -> [0, 1]
    x[9:18] = (x[9:18] + 144) / 100.0

    # SINR: min-max scaling based on observed range
    sinr_min, sinr_max = normalization_info['sinr']['min'], normalization_info['sinr']['max']
    if sinr_max > sinr_min:
        x[18:27] = (x[18:27] - sinr_min) / (sinr_max - sinr_min)
    else:
        # All values identical: set to 0.0
        x[18:27] = 0.0

    # TP: min-max scaling based on observed range
    tp_min, tp_max = normalization_info['tp']['min'], normalization_info['tp']['max']
    if tp_max > tp_min:
        x[27:36] = (x[27:36] - tp_min) / (tp_max - tp_min)
    else:
        x[27:36] = 0.0

    # 5. Debug output
    if debug:
        for i, kpi in enumerate(['CQI', 'RSRP', 'SINR', 'TP']):
            print(f"\n{kpi} percentiles (10th to 90th): {normalization_info[kpi.lower()]['raw_percentiles']}")
            print(f"{kpi} normalization range: min={normalization_info[kpi.lower()]['min']}, max={normalization_info[kpi.lower()]['max']}")
        print(f"\nFinal normalized vector shape: {x.shape}")

    return x

# --- Sample usage example ---
if __name__ == "__main__":
    # Example DataFrame with 5 UEs
    data = {
        'CQI': [5, 10, 12, 7, 15],
        'RSRP': [-120.0, -100.0, -80.0, -110.0, -44.0],
        'SINR': [10.0, 20.0, 15.0, 10.0, 30.0],
        'TP': [5.0, 10.0, 7.5, 6.0, 12.0]
    }
    df = pd.DataFrame(data)
    x = preprocess_kpis(df, debug=True)
    print("\nOutput vector:", x)
    print("Output shape:", x.shape)  # Should be (36,)


CQI percentiles (10th to 90th): [ 5.8  6.6  7.6  8.8 10.  10.8 11.6 12.6 13.8]
CQI normalization range: min=5, max=15

RSRP percentiles (10th to 90th): [-116.  -112.  -108.  -104.  -100.   -92.   -84.   -72.8  -58.4]
RSRP normalization range: min=-120.0, max=-44.0

SINR percentiles (10th to 90th): [10. 10. 11. 13. 15. 17. 19. 22. 26.]
SINR normalization range: min=10.0, max=30.0

TP percentiles (10th to 90th): [ 5.4  5.8  6.3  6.9  7.5  8.5  9.5 10.4 11.2]
TP normalization range: min=5.0, max=12.0

Final normalized vector shape: (36,)

Output vector: [0.38666667 0.44       0.50666667 0.58666667 0.66666667 0.72
 0.77333333 0.84       0.92       0.28       0.32       0.36
 0.4        0.44       0.52       0.6        0.712      0.856
 0.         0.         0.05       0.15       0.25       0.35
 0.45       0.6        0.8        0.05714286 0.11428571 0.18571429
 0.27142857 0.35714286 0.5        0.64285714 0.77142857 0.88571429]
Output shape: (36,)


C:\Users\Jehan_140826\AppData\Local\Temp\1\ipykernel_19904\78697821.py:113: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  x = preprocess_kpis(df, debug=True)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os
from datetime import datetime
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

class MultiFaultDiagnosisNet(nn.Module):
    """
    Feed-forward neural network for multi-label fault diagnosis in SONs.
    Architecture: BN(36) -> FC(84) -> ReLU -> FC(42) -> ReLU -> FC(21) -> ReLU -> FC(3) -> Sigmoid
    """

    def __init__(self, input_dim=36, output_dim=3):
        super(MultiFaultDiagnosisNet, self).__init__()
        # BatchNorm on input
        self.bn = nn.BatchNorm1d(input_dim)
        # Fully connected layers
        self.fc1 = nn.Linear(input_dim, 84)
        self.fc2 = nn.Linear(84, 42)
        self.fc3 = nn.Linear(42, 21)
        self.fc4 = nn.Linear(21, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Input validation: shape and type
        if not torch.is_tensor(x):
            raise ValueError("Input must be a torch.Tensor.")
        if x.ndim != 2 or x.shape[1] != 36:
            raise ValueError(f"Input tensor must have shape (batch_size, 36), got {x.shape}.")
        if not torch.is_floating_point(x):
            raise ValueError("Input tensor must be of floating point type.")
        if not torch.isfinite(x).all():
            raise ValueError("Input contains non-finite values (NaN or Inf).")
        # Forward pass
        x = self.bn(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x  # BCEWithLogitsLoss expects raw logits

    def predict(self, x):
        """
        Predict binary fault labels for input KPI vectors.
        Args:
            x (torch.Tensor): shape (batch_size, 36)
        Returns:
            np.ndarray: shape (batch_size, 3), binary labels (0 or 1)
        """
        self.eval()
        with torch.no_grad():
            logits = self.forward(x)
            probs = torch.sigmoid(logits)
            preds = (probs >= 0.5).int().cpu().numpy()
        return preds

    def save(self, path=None):
        """
        Save model state dict to file with timestamp.
        """
        if path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            path = f"multi_fault_model_{timestamp}.pt"
        torch.save(self.state_dict(), path)
        print(f"Model saved to {path}")

    def load(self, path):
        """
        Load model state dict from file.
        """
        self.load_state_dict(torch.load(path, map_location='cpu'))
        print(f"Model loaded from {path}")

    @staticmethod
    def compute_metrics(y_true, y_pred):
        """
        Compute EMR, macro-F1, recall, precision, and FAR for each fault.
        Args:
            y_true (np.ndarray): shape (n_samples, 3)
            y_pred (np.ndarray): shape (n_samples, 3)
        Returns:
            dict: metrics
        """
        metrics = {}
        # Exact Match Ratio (EMR)
        metrics['EMR'] = np.mean(np.all(y_true == y_pred, axis=1))
        # Macro F1, recall, precision
        metrics['macro_f1'] = f1_score(y_true, y_pred, average='macro', zero_division=0)
        metrics['macro_recall'] = recall_score(y_true, y_pred, average='macro', zero_division=0)
        metrics['macro_precision'] = precision_score(y_true, y_pred, average='macro', zero_division=0)
        # Per-fault FAR: FP / (FP + TN)
        far = []
        for i in range(y_true.shape[1]):
            y_t, y_p = y_true[:, i], y_pred[:, i]
            fp = np.sum((y_p == 1) & (y_t == 0))
            tn = np.sum((y_p == 0) & (y_t == 0))
            far_i = fp / (fp + tn) if (fp + tn) > 0 else 0.0
            far.append(far_i)
        metrics['FAR'] = far
        return metrics

    def train_model(
        self, 
        X_train, y_train, 
        X_val, y_val, 
        epochs=100, 
        batch_size=200, 
        patience=10, 
        verbose=True
    ):
        """
        Train the model with Adam optimizer, BCEWithLogitsLoss, early stopping, and best model saving.
        Args:
            X_train, y_train: training data (numpy arrays)
            X_val, y_val: validation data (numpy arrays)
            epochs: max epochs
            batch_size: batch size (default 200)
            patience: early stopping patience (default 10)
            verbose: print metrics every 10 epochs
        """
        # Input checks
        if X_train.shape[0] == 0 or X_val.shape[0] == 0:
            raise ValueError("Training or validation data is empty.")
        if not np.isfinite(X_train).all() or not np.isfinite(X_val).all():
            raise ValueError("Training or validation data contains non-finite values.")
        if not np.isfinite(y_train).all() or not np.isfinite(y_val).all():
            raise ValueError("Training or validation labels contain non-finite values.")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)
        # Datasets and loaders
        train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
        val_ds = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
        val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)

        optimizer = optim.Adam(self.parameters(), lr=5e-4, betas=(0.9, 0.999))
        criterion = nn.BCEWithLogitsLoss()

        best_emr = -1
        best_state = None
        epochs_no_improve = 0

        for epoch in range(1, epochs + 1):
            self.train()
            train_loss = 0.0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                logits = self.forward(xb)
                # Numerical stability: clamp logits if needed
                loss = criterion(logits, yb)
                if not torch.isfinite(loss):
                    raise ValueError("Non-finite loss encountered during training.")
                loss.backward()
                optimizer.step()
                train_loss += loss.item() * xb.size(0)
            train_loss /= len(train_loader.dataset)

            # Validation
            self.eval()
            val_loss = 0.0
            all_logits, all_labels = [], []
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    logits = self.forward(xb)
                    loss = criterion(logits, yb)
                    val_loss += loss.item() * xb.size(0)
                    all_logits.append(logits.cpu())
                    all_labels.append(yb.cpu())
            val_loss /= len(val_loader.dataset)
            logits = torch.cat(all_logits, dim=0)
            labels = torch.cat(all_labels, dim=0)
            preds = (torch.sigmoid(logits) >= 0.5).int().numpy()
            labels = labels.int().numpy()
            metrics = self.compute_metrics(labels, preds)
            emr = metrics['EMR']

            # Early stopping and best model saving
            if emr > best_emr:
                best_emr = emr
                best_state = self.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
            if verbose and (epoch % 10 == 0 or epoch == 1 or epoch == epochs):
                print(f"Epoch {epoch:3d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, EMR={emr:.4f}, Macro-F1={metrics['macro_f1']:.4f}")
            if epochs_no_improve >= patience:
                if verbose:
                    print(f"Early stopping at epoch {epoch} (no improvement in {patience} epochs).")
                break

        # Restore best model
        if best_state is not None:
            self.load_state_dict(best_state)
            if verbose:
                print(f"Best model restored (EMR={best_emr:.4f}).")
        else:
            print("Warning: No improvement during training.")

# --- Sample training script with synthetic data ---
if __name__ == "__main__":
    np.random.seed(42)
    torch.manual_seed(42)
    # Generate synthetic data: 1000 samples, 36 features, 3 labels
    n_samples = 1000
    X = np.random.rand(n_samples, 36).astype(np.float32)
    # Simulate multi-labels: each fault is present with 30% probability
    y = (np.random.rand(n_samples, 3) < 0.3).astype(np.float32)
    # Split into train/val
    idx = np.random.permutation(n_samples)
    train_idx, val_idx = idx[:800], idx[800:]
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    # Instantiate and train model
    model = MultiFaultDiagnosisNet()
    model.train_model(X_train, y_train, X_val, y_val, epochs=100, batch_size=200, patience=10, verbose=True)

    # Evaluate on validation set
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_pred = model.predict(X_val_tensor)
    metrics = model.compute_metrics(y_val, y_pred)
    print("\nValidation metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v}")

    # Save and load model
    model.save()
    # model.load('multi_fault_model_YYYYMMDD_HHMMSS.pt')  # Example usage

Epoch   1: Train Loss=0.6671, Val Loss=0.6635, EMR=0.3950, Macro-F1=0.0000
Epoch  10: Train Loss=0.6282, Val Loss=0.6151, EMR=0.3950, Macro-F1=0.0000
Early stopping at epoch 11 (no improvement in 10 epochs).
Best model restored (EMR=0.3950).

Validation metrics:
EMR: 0.395
macro_f1: 0.0
macro_recall: 0.0
macro_precision: 0.0
FAR: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Model saved to multi_fault_model_20250617_135153.pt


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
import warnings
from datetime import datetime

# ------------------- Fault Diagnosis Neural Network -------------------
class FaultDiagnosisNN(nn.Module):
    """
    Fault diagnosis neural network:
    BN(36) -> FC(64) -> ReLU -> FC(32) -> ReLU -> FC(18) -> ReLU -> FC(3) -> Sigmoid
    Multi-label output for ERP, ED, EU.
    """
    def __init__(self, input_dim=36, output_dim=3):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 18)
        self.fc4 = nn.Linear(18, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Input validation
        if not torch.is_tensor(x):
            raise ValueError("Input must be a torch.Tensor.")
        if x.ndim != 2 or x.shape[1] != 36:
            raise ValueError(f"Input tensor must have shape (batch_size, 36), got {x.shape}.")
        if not torch.is_floating_point(x):
            raise ValueError("Input tensor must be of floating point type.")
        if not torch.isfinite(x).all():
            raise ValueError("Input contains non-finite values (NaN or Inf).")
        x = self.bn(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x  # BCEWithLogitsLoss expects raw logits

# ------------------- Severity Diagnosis Neural Network -------------------
class SeverityDiagnosisNN(nn.Module):
    """
    Severity diagnosis neural network (for one fault):
    BN(36) -> FC(16) -> ReLU -> FC(9) -> Softmax
    Output: 3 severity levels (slight, medium, severe)
    """
    def __init__(self, input_dim=36, output_dim=3):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 9)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Input validation
        if not torch.is_tensor(x):
            raise ValueError("Input must be a torch.Tensor.")
        if x.ndim != 2 or x.shape[1] != 36:
            raise ValueError(f"Input tensor must have shape (batch_size, 36), got {x.shape}.")
        if not torch.is_floating_point(x):
            raise ValueError("Input tensor must be of floating point type.")
        if not torch.isfinite(x).all():
            raise ValueError("Input contains non-finite values (NaN or Inf).")
        x = self.bn(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # CrossEntropyLoss expects raw logits

# ------------------- Composite Model -------------------
class SeparatedSeverityDiagnosisModel(nn.Module):
    """
    Composite model: combines one fault diagnosis NN and three severity NNs.
    """
    def __init__(self):
        super().__init__()
        self.fault_nn = FaultDiagnosisNN()
        self.severity_nns = nn.ModuleList([SeverityDiagnosisNN() for _ in range(3)])

    def forward(self, x):
        # Returns: fault_logits (batch, 3), severity_logits (batch, 9)
        fault_logits = self.fault_nn(x)
        severity_logits = [net(x) for net in self.severity_nns]  # Each: (batch, 3)
        severity_logits = torch.cat(severity_logits, dim=1)      # (batch, 9)
        return fault_logits, severity_logits

    def predict(self, x):
        """
        Predicts faults and severity levels.
        Returns:
            faults: (batch, 3) binary (0/1)
            severity: (batch, 9) one-hot per fault (3 per fault)
        """
        self.eval()
        with torch.no_grad():
            fault_logits, severity_logits = self.forward(x)
            faults = (torch.sigmoid(fault_logits) >= 0.5).int()
            # For each fault, select argmax severity (one-hot)
            severity_preds = []
            for i in range(3):
                logits = severity_logits[:, i*3:(i+1)*3]
                idx = torch.argmax(logits, dim=1)
                one_hot = torch.zeros_like(logits)
                one_hot[torch.arange(logits.size(0)), idx] = 1
                severity_preds.append(one_hot)
            severity = torch.cat(severity_preds, dim=1).int()
            # Contradiction check: ED+EU both predicted
            contradiction = ((faults[:,1] == 1) & (faults[:,2] == 1)).any()
            if contradiction:
                warnings.warn("ED and EU both predicted for at least one sample (contradiction).")
        return faults.cpu().numpy(), severity.cpu().numpy()

    def save(self, path=None):
        if path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            path = f"separated_severity_model_{timestamp}.pt"
        torch.save(self.state_dict(), path)
        print(f"Model saved to {path}")

    def load(self, path):
        self.load_state_dict(torch.load(path, map_location='cpu'))
        print(f"Model loaded from {path}")

    @staticmethod
    def compute_metrics(y_fault_true, y_fault_pred, y_sev_true, y_sev_pred):
        """
        Compute metrics for fault and severity diagnosis.
        Returns: dict with 4-decimal precision.
        """
        metrics = {}
        # Fault metrics
        metrics['EMR_F'] = float(np.mean(np.all(y_fault_true == y_fault_pred, axis=1)))
        metrics['macro-F1_F'] = float(f1_score(y_fault_true, y_fault_pred, average='macro', zero_division=0))
        metrics['macro_recall_F'] = float(recall_score(y_fault_true, y_fault_pred, average='macro', zero_division=0))
        metrics['macro_precision_F'] = float(precision_score(y_fault_true, y_fault_pred, average='macro', zero_division=0))
        # FAR per fault
        far_f = []
        for i in range(3):
            y_t, y_p = y_fault_true[:, i], y_fault_pred[:, i]
            fp = np.sum((y_p == 1) & (y_t == 0))
            tn = np.sum((y_p == 0) & (y_t == 0))
            far_i = fp / (fp + tn) if (fp + tn) > 0 else 0.0
            far_f.append(round(far_i, 4))
        metrics['FAR_F'] = far_f

        # Severity metrics (flattened per-fault one-hot to class index)
        n = y_sev_true.shape[0]
        y_true_idx = np.argmax(y_sev_true.reshape(n, 3, 3), axis=2)
        y_pred_idx = np.argmax(y_sev_pred.reshape(n, 3, 3), axis=2)
        # EMR_S: all 3 severities match
        metrics['EMR_S'] = float(np.mean(np.all(y_true_idx == y_pred_idx, axis=1)))
        # Macro-F1, recall, precision for all severity levels (SLT, MED, SEV)
        f1s, recs, precs, fars = [], [], [], []
        for sev in range(3):  # 0:SLT, 1:MED, 2:SEV
            y_true_flat = (y_true_idx == sev).astype(int).flatten()
            y_pred_flat = (y_pred_idx == sev).astype(int).flatten()
            f1s.append(f1_score(y_true_flat, y_pred_flat, zero_division=0))
            recs.append(recall_score(y_true_flat, y_pred_flat, zero_division=0))
            precs.append(precision_score(y_true_flat, y_pred_flat, zero_division=0))
            fp = np.sum((y_pred_flat == 1) & (y_true_flat == 0))
            tn = np.sum((y_pred_flat == 0) & (y_true_flat == 0))
            far = fp / (fp + tn) if (fp + tn) > 0 else 0.0
            fars.append(round(far, 4))
        metrics['macro-F1_S'] = float(np.mean(f1s))
        metrics['macro_recall_S'] = float(np.mean(recs))
        metrics['macro_precision_S'] = float(np.mean(precs))
        metrics['FAR_S'] = fars
        # Round all float metrics to 4 decimals
        for k in metrics:
            if isinstance(metrics[k], float):
                metrics[k] = round(metrics[k], 4)
        return metrics

    def train_model(
        self, 
        X_train, y_fault_train, y_sev_train, 
        X_val, y_fault_val, y_sev_val, 
        epochs=100, batch_size=200, patience=10, verbose=True
    ):
        """
        Trains the composite model with separate optimizers and losses.
        Early stopping on combined validation loss.
        """
        # Input checks
        if X_train.shape[0] == 0 or X_val.shape[0] == 0:
            raise ValueError("Training or validation data is empty.")
        if not np.isfinite(X_train).all() or not np.isfinite(X_val).all():
            raise ValueError("Training or validation data contains non-finite values.")
        if not np.isfinite(y_fault_train).all() or not np.isfinite(y_fault_val).all():
            raise ValueError("Fault labels contain non-finite values.")
        if not np.isfinite(y_sev_train).all() or not np.isfinite(y_sev_val).all():
            raise ValueError("Severity labels contain non-finite values.")
        if y_fault_train.shape[1] != 3 or y_sev_train.shape[1] != 9:
            raise ValueError("Fault labels must have shape (batch, 3), severity labels (batch, 9).")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)
        # Datasets and loaders
        train_ds = TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_fault_train, dtype=torch.float32),
            torch.tensor(y_sev_train, dtype=torch.float32)
        )
        val_ds = TensorDataset(
            torch.tensor(X_val, dtype=torch.float32),
            torch.tensor(y_fault_val, dtype=torch.float32),
            torch.tensor(y_sev_val, dtype=torch.float32)
        )
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
        val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)

        # Optimizers
        opt_fault = optim.Adam(self.fault_nn.parameters(), lr=5e-4)
        opt_sev = optim.Adam(self.severity_nns.parameters(), lr=2.5e-3)
        # Losses
        loss_fault = nn.BCEWithLogitsLoss()
        loss_sev = nn.CrossEntropyLoss()

        best_emr_s = -1
        best_state = None
        epochs_no_improve = 0

        for epoch in range(1, epochs + 1):
            self.train()
            train_loss = 0.0
            for xb, yb_fault, yb_sev in train_loader:
                xb = xb.to(device)
                yb_fault = yb_fault.to(device)
                yb_sev = yb_sev.to(device)
                # Forward
                fault_logits, severity_logits = self.forward(xb)
                # Fault loss
                lf = loss_fault(fault_logits, yb_fault)
                # Severity loss: for each fault, CrossEntropyLoss
                ls = 0.0
                for i in range(3):
                    # yb_sev: (batch, 9) -> (batch, 3) one-hot for fault i
                    y_true = yb_sev[:, i*3:(i+1)*3]
                    # Convert one-hot to class index
                    y_idx = torch.argmax(y_true, dim=1)
                    ls += loss_sev(severity_logits[:, i*3:(i+1)*3], y_idx)
                ls /= 3.0
                loss = lf + ls
                if not torch.isfinite(loss):
                    raise ValueError("Non-finite loss encountered during training.")
                # Backprop
                opt_fault.zero_grad()
                opt_sev.zero_grad()
                loss.backward()
                opt_fault.step()
                opt_sev.step()
                train_loss += loss.item() * xb.size(0)
            train_loss /= len(train_loader.dataset)

            # Validation
            self.eval()
            val_loss = 0.0
            all_fault_logits, all_sev_logits, all_fault_labels, all_sev_labels = [], [], [], []
            with torch.no_grad():
                for xb, yb_fault, yb_sev in val_loader:
                    xb = xb.to(device)
                    yb_fault = yb_fault.to(device)
                    yb_sev = yb_sev.to(device)
                    fault_logits, severity_logits = self.forward(xb)
                    lf = loss_fault(fault_logits, yb_fault)
                    ls = 0.0
                    for i in range(3):
                        y_true = yb_sev[:, i*3:(i+1)*3]
                        y_idx = torch.argmax(y_true, dim=1)
                        ls += loss_sev(severity_logits[:, i*3:(i+1)*3], y_idx)
                    ls /= 3.0
                    loss = lf + ls
                    val_loss += loss.item() * xb.size(0)
                    all_fault_logits.append(fault_logits.cpu())
                    all_sev_logits.append(severity_logits.cpu())
                    all_fault_labels.append(yb_fault.cpu())
                    all_sev_labels.append(yb_sev.cpu())
            val_loss /= len(val_loader.dataset)
            # Predictions and metrics
            fault_logits = torch.cat(all_fault_logits, dim=0)
            sev_logits = torch.cat(all_sev_logits, dim=0)
            fault_labels = torch.cat(all_fault_labels, dim=0)
            sev_labels = torch.cat(all_sev_labels, dim=0)
            fault_preds = (torch.sigmoid(fault_logits) >= 0.5).int().numpy()
            sev_preds = []
            for i in range(3):
                logits = sev_logits[:, i*3:(i+1)*3]
                idx = torch.argmax(logits, dim=1)
                one_hot = torch.zeros_like(logits)
                one_hot[torch.arange(logits.size(0)), idx] = 1
                sev_preds.append(one_hot)
            sev_preds = torch.cat(sev_preds, dim=1).int().numpy()
            metrics = self.compute_metrics(
                fault_labels.int().numpy(), fault_preds,
                sev_labels.int().numpy(), sev_preds
            )
            emr_s = metrics['EMR_S']

            # Early stopping and best model saving
            if emr_s > best_emr_s:
                best_emr_s = emr_s
                best_state = self.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
            if verbose and (epoch % 10 == 0 or epoch == 1 or epoch == epochs):
                print(f"Epoch {epoch:3d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, EMR_S={emr_s:.4f}, Macro-F1_S={metrics['macro-F1_S']:.4f}")
            if epochs_no_improve >= patience:
                if verbose:
                    print(f"Early stopping at epoch {epoch} (no improvement in {patience} epochs).")
                break

        # Restore best model
        if best_state is not None:
            self.load_state_dict(best_state)
            if verbose:
                print(f"Best model restored (EMR_S={best_emr_s:.4f}).")
        else:
            print("Warning: No improvement during training.")

# ------------------- Sample Training Script -------------------
if __name__ == "__main__":
    np.random.seed(42)
    torch.manual_seed(42)
    # Synthetic data: 1000 samples, 36 features, 3 faults, 9 severity (3 per fault, one-hot)
    n_samples = 1000
    X = np.random.rand(n_samples, 36).astype(np.float32)
    # Fault labels: each fault present with 30% probability
    y_fault = (np.random.rand(n_samples, 3) < 0.3).astype(np.float32)
    # Severity labels: for each fault, one-hot (random class)
    y_sev = np.zeros((n_samples, 9), dtype=np.float32)
    for i in range(3):
        idx = np.random.randint(0, 3, size=n_samples)
        y_sev[np.arange(n_samples), i*3 + idx] = 1.0
    # Split train/val/test
    idx = np.random.permutation(n_samples)
    train_idx, val_idx, test_idx = idx[:700], idx[700:850], idx[850:]
    X_train, y_fault_train, y_sev_train = X[train_idx], y_fault[train_idx], y_sev[train_idx]
    X_val, y_fault_val, y_sev_val = X[val_idx], y_fault[val_idx], y_sev[val_idx]
    X_test, y_fault_test, y_sev_test = X[test_idx], y_fault[test_idx], y_sev[test_idx]

    # Instantiate and train model
    model = SeparatedSeverityDiagnosisModel()
    model.train_model(
        X_train, y_fault_train, y_sev_train,
        X_val, y_fault_val, y_sev_val,
        epochs=100, batch_size=200, patience=10, verbose=True
    )

    # Evaluate on test set
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_fault_pred, y_sev_pred = model.predict(X_test_tensor)
    metrics = model.compute_metrics(y_fault_test, y_fault_pred, y_sev_test, y_sev_pred)
    print("\nTest set metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v}")

    # Save and load model
    model.save()
    # model.load('separated_severity_model_YYYYMMDD_HHMMSS.pt')  # Example usage

Epoch   1: Train Loss=1.7997, Val Loss=1.8005, EMR_S=0.0267, Macro-F1_S=0.3090
Epoch  10: Train Loss=1.7173, Val Loss=1.7549, EMR_S=0.0333, Macro-F1_S=0.3120
Early stopping at epoch 15 (no improvement in 10 epochs).
Best model restored (EMR_S=0.0467).

Test set metrics:
EMR_F: 0.3333
macro-F1_F: 0.0
macro_recall_F: 0.0
macro_precision_F: 0.0
FAR_F: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
EMR_S: 0.02
macro-F1_S: 0.3043
macro_recall_S: 0.308
macro_precision_S: 0.3122
FAR_S: [np.float64(0.4194), np.float64(0.2518), np.float64(0.3636)]
Model saved to separated_severity_model_20250617_135154.pt


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score
import warnings
from datetime import datetime

# ------------------- Neural Network Definition -------------------
class FaultDiagnosisNet(nn.Module):
    """
    Feed-forward neural network for multi-label fault diagnosis.
    Architecture: BN(36) -> FC(84) -> ReLU -> FC(42) -> ReLU -> FC(21) -> ReLU -> FC(3)
    """
    def __init__(self, input_dim=36, output_dim=3):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 84)
        self.fc2 = nn.Linear(84, 42)
        self.fc3 = nn.Linear(42, 21)
        self.fc4 = nn.Linear(21, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, return_features=False):
        # Input validation
        if not torch.is_tensor(x):
            raise ValueError("Input must be a torch.Tensor.")
        if x.ndim != 2 or x.shape[1] != 36:
            raise ValueError(f"Input tensor must have shape (batch_size, 36), got {x.shape}.")
        if not torch.is_floating_point(x):
            raise ValueError("Input tensor must be of floating point type.")
        if not torch.isfinite(x).all():
            raise ValueError("Input contains non-finite values (NaN or Inf).")
        x = self.bn(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        features = self.relu(self.fc3(x))  # Features for MMD (size 21)
        logits = self.fc4(features)
        if return_features:
            return logits, features
        return logits

# ------------------- MMD Loss Function -------------------
def gaussian_kernel(x, y, bandwidths):
    """
    Computes a multi-kernel Gaussian kernel matrix between x and y.
    Args:
        x: (n, d)
        y: (m, d)
        bandwidths: list of kernel bandwidths
    Returns:
        (n, m) kernel matrix
    """
    xx = x.unsqueeze(1)  # (n, 1, d)
    yy = y.unsqueeze(0)  # (1, m, d)
    L2_dist = ((xx - yy) ** 2).sum(2)  # (n, m)
    kernels = [torch.exp(-L2_dist / (2 * bw ** 2)) for bw in bandwidths]
    return sum(kernels) / len(kernels)

def mmd_loss(source, target, bandwidths=[0.1, 1, 10]):
    """
    Computes Maximum Mean Discrepancy (MMD) between source and target features.
    Uses multiple Gaussian kernels for robustness.
    MMD = sqrt(||E[φ(X_s)] - E[φ(X_t)]||^2)
    """
    n, m = source.size(0), target.size(0)
    if n == 0 or m == 0:
        return torch.tensor(0.0, device=source.device)
    K_ss = gaussian_kernel(source, source, bandwidths)
    K_tt = gaussian_kernel(target, target, bandwidths)
    K_st = gaussian_kernel(source, target, bandwidths)
    mmd = K_ss.mean() + K_tt.mean() - 2 * K_st.mean()
    mmd = torch.sqrt(torch.clamp(mmd, min=0.0))  # Ensure non-negative
    return mmd

# ------------------- Regularization Loss -------------------
def l2_regularization(model):
    """
    Computes L2 norm of all model parameters.
    """
    l2 = torch.tensor(0.0, device=next(model.parameters()).device)
    for p in model.parameters():
        l2 += torch.norm(p, 2)
    return l2

# ------------------- Evaluation Metrics -------------------
def compute_metrics(y_true, y_pred):
    """
    Computes EMR, macro-F1, recall, precision, FAR for multi-label classification.
    """
    metrics = {}
    metrics['EMR'] = float(np.mean(np.all(y_true == y_pred, axis=1)))
    metrics['macro-F1'] = float(f1_score(y_true, y_pred, average='macro', zero_division=0))
    metrics['macro_recall'] = float(recall_score(y_true, y_pred, average='macro', zero_division=0))
    metrics['macro_precision'] = float(precision_score(y_true, y_pred, average='macro', zero_division=0))
    # FAR per fault
    far = []
    for i in range(y_true.shape[1]):
        y_t, y_p = y_true[:, i], y_pred[:, i]
        fp = np.sum((y_p == 1) & (y_t == 0))
        tn = np.sum((y_p == 0) & (y_t == 0))
        far_i = fp / (fp + tn) if (fp + tn) > 0 else 0.0
        far.append(round(far_i, 4))
    metrics['FAR'] = far
    # Round all float metrics to 4 decimals
    for k in metrics:
        if isinstance(metrics[k], float):
            metrics[k] = round(metrics[k], 4)
    return metrics

# ------------------- Training Loop -------------------
def train_transfer_model(
    model, 
    source_loader, target_loader, 
    val_loader, 
    epochs=100, 
    lambda_mmd=1.0, 
    lambda_reg=0.01, 
    lr=5e-4, 
    patience=10, 
    verbose=True
):
    """
    Trains the model with unsupervised transfer learning using MMD loss.
    Alternates between source and target batches.
    Early stopping on source validation EMR.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    best_emr = -1
    best_state = None
    epochs_no_improve = 0

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        source_iter = iter(source_loader)
        target_iter = iter(target_loader)
        n_batches = min(len(source_loader), len(target_loader))
        for _ in range(n_batches):
            # Get source batch
            try:
                xb_s, yb_s = next(source_iter)
            except StopIteration:
                continue
            xb_s, yb_s = xb_s.to(device), yb_s.to(device)
            # Get target batch
            try:
                xb_t = next(target_iter)[0]
            except StopIteration:
                continue
            xb_t = xb_t.to(device)
            # Forward: source
            logits_s, feat_s = model(xb_s, return_features=True)
            # Forward: target
            _, feat_t = model(xb_t, return_features=True)
            # Losses
            cls_loss = criterion(logits_s, yb_s)
            mmd = mmd_loss(feat_s, feat_t)
            reg = l2_regularization(model)
            total_loss = cls_loss + lambda_mmd * mmd + lambda_reg * reg
            if not torch.isfinite(total_loss):
                raise ValueError("Non-finite loss encountered during training.")
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            train_loss += total_loss.item() * xb_s.size(0)
        if n_batches > 0:
            train_loss /= (n_batches * source_loader.batch_size)
        else:
            train_loss = 0.0

        # Validation on source domain
        model.eval()
        all_logits, all_labels = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                all_logits.append(logits.cpu())
                all_labels.append(yb.cpu())
        if all_logits:
            logits = torch.cat(all_logits, dim=0)
            labels = torch.cat(all_labels, dim=0)
            preds = (torch.sigmoid(logits) >= 0.5).int().numpy()
            labels = labels.int().numpy()
            metrics = compute_metrics(labels, preds)
            emr = metrics['EMR']
            # Contradiction check: ED+EU both predicted
            contradiction = ((preds[:,1] == 1) & (preds[:,2] == 1)).any()
            if contradiction:
                warnings.warn("ED and EU both predicted for at least one sample (contradiction).")
        else:
            metrics = {}
            emr = 0.0

        # Early stopping and best model saving
        if emr > best_emr:
            best_emr = emr
            best_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if verbose and (epoch % 10 == 0 or epoch == 1 or epoch == epochs):
            print(f"Epoch {epoch:3d}: Train Loss={train_loss:.4f}, Val EMR={emr:.4f}, Macro-F1={metrics.get('macro-F1', 0):.4f}")
        if epochs_no_improve >= patience:
            if verbose:
                print(f"Early stopping at epoch {epoch} (no improvement in {patience} epochs).")
            break

    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)
        if verbose:
            print(f"Best model restored (EMR={best_emr:.4f}).")
    else:
        print("Warning: No improvement during training.")

# ------------------- Model Save/Load -------------------
def save_model(model, path=None):
    if path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        path = f"mmd_transfer_model_{timestamp}.pt"
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path, map_location='cpu'))
    print(f"Model loaded from {path}")

# ------------------- Sample Script with Synthetic Data -------------------
if __name__ == "__main__":
    np.random.seed(42)
    torch.manual_seed(42)
    # Synthetic source data: 1000 samples, 36 features, 3 labels
    n_source = 1000
    X_source = np.random.rand(n_source, 36).astype(np.float32)
    y_source = (np.random.rand(n_source, 3) < 0.3).astype(np.float32)
    # Synthetic target data: 1000 samples, 36 features, no labels
    n_target = 1000
    X_target = np.random.rand(n_target, 36).astype(np.float32)
    # Split source into train/val
    idx = np.random.permutation(n_source)
    train_idx, val_idx = idx[:800], idx[800:]
    X_train, y_train = X_source[train_idx], y_source[train_idx]
    X_val, y_val = X_source[val_idx], y_source[val_idx]
    # Target train only (no val)
    X_target_train = X_target[:800]

    # DataLoaders
    batch_size = 200
    source_train_loader = DataLoader(TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32)
    ), batch_size=batch_size, shuffle=True, drop_last=True)
    source_val_loader = DataLoader(TensorDataset(
        torch.tensor(X_val, dtype=torch.float32),
        torch.tensor(y_val, dtype=torch.float32)
    ), batch_size=batch_size, shuffle=False, drop_last=False)
    target_train_loader = DataLoader(TensorDataset(
        torch.tensor(X_target_train, dtype=torch.float32),
    ), batch_size=batch_size, shuffle=True, drop_last=True)

    # Instantiate and train model
    model = FaultDiagnosisNet()
    train_transfer_model(
        model, 
        source_train_loader, 
        target_train_loader, 
        source_val_loader, 
        epochs=100, 
        lambda_mmd=1.0, 
        lambda_reg=0.01, 
        lr=5e-4, 
        patience=10, 
        verbose=True
    )

    # Evaluate on source validation set
    model.eval()
    all_logits, all_labels = [], []
    with torch.no_grad():
        for xb, yb in source_val_loader:
            logits = model(xb)
            all_logits.append(logits.cpu())
            all_labels.append(yb.cpu())
    logits = torch.cat(all_logits, dim=0)
    labels = torch.cat(all_labels, dim=0)
    preds = (torch.sigmoid(logits) >= 0.5).int().numpy()
    labels = labels.int().numpy()
    metrics = compute_metrics(labels, preds)
    print("\nSource validation metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v}")

    # Save model
    save_model(model)
    # To load: load_model(model, 'mmd_transfer_model_YYYYMMDD_HHMMSS.pt')

Epoch   1: Train Loss=0.9254, Val EMR=0.3300, Macro-F1=0.0000
Epoch  10: Train Loss=0.8985, Val EMR=0.3300, Macro-F1=0.0000
Early stopping at epoch 11 (no improvement in 10 epochs).
Best model restored (EMR=0.3300).

Source validation metrics:
EMR: 0.33
macro-F1: 0.0
macro_recall: 0.0
macro_precision: 0.0
FAR: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Model saved to mmd_transfer_model_20250617_135156.pt


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score
import warnings
from datetime import datetime

# ------------------- Neural Network Definition -------------------
class FaultDiagnosisNet(nn.Module):
    """
    Feed-forward neural network for multi-label fault diagnosis.
    Architecture: BN(36) -> FC(84) -> ReLU -> FC(42) -> ReLU -> FC(21) -> ReLU -> FC(3)
    """
    def __init__(self, input_dim=36, output_dim=3):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 84)
        self.fc2 = nn.Linear(84, 42)
        self.fc3 = nn.Linear(42, 21)
        self.fc4 = nn.Linear(21, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, return_features=False):
        # Input validation
        if not torch.is_tensor(x):
            raise ValueError("Input must be a torch.Tensor.")
        if x.ndim != 2 or x.shape[1] != 36:
            raise ValueError(f"Input tensor must have shape (batch_size, 36), got {x.shape}.")
        if not torch.is_floating_point(x):
            raise ValueError("Input tensor must be of floating point type.")
        if not torch.isfinite(x).all():
            raise ValueError("Input contains non-finite values (NaN or Inf).")
        x = self.bn(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        features = self.relu(self.fc3(x))  # Features for CORAL (size 21)
        logits = self.fc4(features)
        if return_features:
            return logits, features
        return logits

# ------------------- CORAL Loss Function -------------------
def coral_loss(source, target, epsilon=1e-6):
    """
    Computes CORAL loss between source and target features.
    CORAL = ||Cov(X_s) - Cov(X_t)||_F^2 / (4 * d^2)
    Cov(X) = (X^T X - (1/n) * (1^T X)^T (1^T X)) / (n-1)
    """
    d = source.size(1)
    n_s = source.size(0)
    n_t = target.size(0)
    # Center features
    source_mean = source.mean(dim=0, keepdim=True)
    target_mean = target.mean(dim=0, keepdim=True)
    source_c = source - source_mean
    target_c = target - target_mean
    # Covariance matrices
    cov_s = (source_c.t() @ source_c) / (n_s - 1)
    cov_t = (target_c.t() @ target_c) / (n_t - 1)
    # Add epsilon to diagonal for numerical stability
    cov_s += torch.eye(d, device=source.device) * epsilon
    cov_t += torch.eye(d, device=target.device) * epsilon
    # Frobenius norm squared
    loss = torch.sum((cov_s - cov_t) ** 2)
    loss = loss / (4 * d * d)
    return loss

# ------------------- Regularization Loss -------------------
def l2_regularization(model):
    """
    Computes L2 norm of all model parameters.
    """
    l2 = torch.tensor(0.0, device=next(model.parameters()).device)
    for p in model.parameters():
        l2 += torch.norm(p, 2)
    return l2

# ------------------- Evaluation Metrics -------------------
def compute_metrics(y_true, y_pred):
    """
    Computes EMR, macro-F1, recall, precision, FAR for multi-label classification.
    """
    metrics = {}
    metrics['EMR'] = float(np.mean(np.all(y_true == y_pred, axis=1)))
    metrics['macro-F1'] = float(f1_score(y_true, y_pred, average='macro', zero_division=0))
    metrics['macro_recall'] = float(recall_score(y_true, y_pred, average='macro', zero_division=0))
    metrics['macro_precision'] = float(precision_score(y_true, y_pred, average='macro', zero_division=0))
    # FAR per fault
    far = []
    for i in range(y_true.shape[1]):
        y_t, y_p = y_true[:, i], y_pred[:, i]
        fp = np.sum((y_p == 1) & (y_t == 0))
        tn = np.sum((y_p == 0) & (y_t == 0))
        far_i = fp / (fp + tn) if (fp + tn) > 0 else 0.0
        far.append(round(far_i, 4))
    metrics['FAR'] = far
    # Round all float metrics to 4 decimals
    for k in metrics:
        if isinstance(metrics[k], float):
            metrics[k] = round(metrics[k], 4)
    return metrics

# ------------------- Training Loop -------------------
def train_transfer_model(
    model, 
    source_loader, target_loader, 
    val_loader, 
    epochs=100, 
    lambda_coral=1.0, 
    lambda_reg=0.01, 
    lr=5e-4, 
    patience=10, 
    verbose=True
):
    """
    Trains the model with unsupervised transfer learning using CORAL loss.
    Alternates between source and target batches.
    Early stopping on source validation EMR.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    best_emr = -1
    best_state = None
    epochs_no_improve = 0

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        source_iter = iter(source_loader)
        target_iter = iter(target_loader)
        n_batches = min(len(source_loader), len(target_loader))
        for _ in range(n_batches):
            # Get source batch
            try:
                xb_s, yb_s = next(source_iter)
            except StopIteration:
                continue
            xb_s, yb_s = xb_s.to(device), yb_s.to(device)
            # Get target batch
            try:
                xb_t = next(target_iter)[0]
            except StopIteration:
                continue
            xb_t = xb_t.to(device)
            # Forward: source
            logits_s, feat_s = model(xb_s, return_features=True)
            # Forward: target
            _, feat_t = model(xb_t, return_features=True)
            # Losses
            cls_loss = criterion(logits_s, yb_s)
            coral = coral_loss(feat_s, feat_t)
            reg = l2_regularization(model)
            total_loss = cls_loss + lambda_coral * coral + lambda_reg * reg
            if not torch.isfinite(total_loss):
                raise ValueError("Non-finite loss encountered during training.")
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            train_loss += total_loss.item() * xb_s.size(0)
        if n_batches > 0:
            train_loss /= (n_batches * source_loader.batch_size)
        else:
            train_loss = 0.0

        # Validation on source domain
        model.eval()
        all_logits, all_labels = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                all_logits.append(logits.cpu())
                all_labels.append(yb.cpu())
        if all_logits:
            logits = torch.cat(all_logits, dim=0)
            labels = torch.cat(all_labels, dim=0)
            preds = (torch.sigmoid(logits) >= 0.5).int().numpy()
            labels = labels.int().numpy()
            metrics = compute_metrics(labels, preds)
            emr = metrics['EMR']
            # Contradiction check: ED+EU both predicted
            contradiction = ((preds[:,1] == 1) & (preds[:,2] == 1)).any()
            if contradiction:
                warnings.warn("ED and EU both predicted for at least one sample (contradiction).")
        else:
            metrics = {}
            emr = 0.0

        # Early stopping and best model saving
        if emr > best_emr:
            best_emr = emr
            best_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if verbose and (epoch % 10 == 0 or epoch == 1 or epoch == epochs):
            print(f"Epoch {epoch:3d}: Train Loss={train_loss:.4f}, Val EMR={emr:.4f}, Macro-F1={metrics.get('macro-F1', 0):.4f}")
        if epochs_no_improve >= patience:
            if verbose:
                print(f"Early stopping at epoch {epoch} (no improvement in {patience} epochs).")
            break

    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)
        if verbose:
            print(f"Best model restored (EMR={best_emr:.4f}).")
    else:
        print("Warning: No improvement during training.")

# ------------------- Model Save/Load -------------------
def save_model(model, path=None):
    if path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        path = f"coral_transfer_model_{timestamp}.pt"
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path, map_location='cpu'))
    print(f"Model loaded from {path}")

# ------------------- Sample Script with Synthetic Data -------------------
if __name__ == "__main__":
    np.random.seed(42)
    torch.manual_seed(42)
    # Synthetic source data: 1000 samples, 36 features, 3 labels
    n_source = 1000
    X_source = np.random.rand(n_source, 36).astype(np.float32)
    y_source = (np.random.rand(n_source, 3) < 0.3).astype(np.float32)
    # Synthetic target data: 1000 samples, 36 features, no labels
    n_target = 1000
    X_target = np.random.rand(n_target, 36).astype(np.float32)
    # Split source into train/val
    idx = np.random.permutation(n_source)
    train_idx, val_idx = idx[:800], idx[800:]
    X_train, y_train = X_source[train_idx], y_source[train_idx]
    X_val, y_val = X_source[val_idx], y_source[val_idx]
    # Target train only (no val)
    X_target_train = X_target[:800]

    # DataLoaders
    batch_size = 200
    source_train_loader = DataLoader(TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32)
    ), batch_size=batch_size, shuffle=True, drop_last=True)
    source_val_loader = DataLoader(TensorDataset(
        torch.tensor(X_val, dtype=torch.float32),
        torch.tensor(y_val, dtype=torch.float32)
    ), batch_size=batch_size, shuffle=False, drop_last=False)
    target_train_loader = DataLoader(TensorDataset(
        torch.tensor(X_target_train, dtype=torch.float32),
    ), batch_size=batch_size, shuffle=True, drop_last=True)

    # Instantiate and train model
    model = FaultDiagnosisNet()
    train_transfer_model(
        model, 
        source_train_loader, 
        target_train_loader, 
        source_val_loader, 
        epochs=100, 
        lambda_coral=1.0, 
        lambda_reg=0.01, 
        lr=5e-4, 
        patience=10, 
        verbose=True
    )

    # Evaluate on source validation set
    model.eval()
    all_logits, all_labels = [], []
    with torch.no_grad():
        for xb, yb in source_val_loader:
            logits = model(xb)
            all_logits.append(logits.cpu())
            all_labels.append(yb.cpu())
    logits = torch.cat(all_logits, dim=0)
    labels = torch.cat(all_labels, dim=0)
    preds = (torch.sigmoid(logits) >= 0.5).int().numpy()
    labels = labels.int().numpy()
    metrics = compute_metrics(labels, preds)
    print("\nSource validation metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v}")

    # Save model
    save_model(model)
    # To load: load_model(model, 'coral_transfer_model_YYYYMMDD_HHMMSS.pt')

Epoch   1: Train Loss=0.8711, Val EMR=0.3300, Macro-F1=0.0000
Epoch  10: Train Loss=0.8248, Val EMR=0.3300, Macro-F1=0.0000
Early stopping at epoch 11 (no improvement in 10 epochs).
Best model restored (EMR=0.3300).

Source validation metrics:
EMR: 0.33
macro-F1: 0.0
macro_recall: 0.0
macro_precision: 0.0
FAR: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Model saved to coral_transfer_model_20250617_135157.pt


In [11]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score

def evaluate_diagnosis_performance(
    y_fault_true, y_fault_pred_prob,
    y_sev_true, y_sev_pred_prob
):
    """
    Evaluate multi-fault and severity diagnosis performance as per 2024 IEEE paper.

    Parameters:
        y_fault_true: np.ndarray, shape (n_samples, 3), binary (0/1)
        y_fault_pred_prob: np.ndarray, shape (n_samples, 3), [0,1]
        y_sev_true: np.ndarray, shape (n_samples, 9), binary (0/1), at most one 1 per fault
        y_sev_pred_prob: np.ndarray, shape (n_samples, 9), [0,1]

    Returns:
        dict with EMR_F, EMR_S, macro-F1_F, macro-F1_S, per-fault and per-severity-level metrics
    """

    # ------------------ Input Validation ------------------
    n_samples = y_fault_true.shape[0]
    # Check shapes
    if y_fault_true.shape != (n_samples, 3):
        raise ValueError("y_fault_true must have shape (n_samples, 3)")
    if y_fault_pred_prob.shape != (n_samples, 3):
        raise ValueError("y_fault_pred_prob must have shape (n_samples, 3)")
    if y_sev_true.shape != (n_samples, 9):
        raise ValueError("y_sev_true must have shape (n_samples, 9)")
    if y_sev_pred_prob.shape != (n_samples, 9):
        raise ValueError("y_sev_pred_prob must have shape (n_samples, 9)")
    # Check binary true labels
    if not np.all(np.isin(y_fault_true, [0, 1])):
        raise ValueError("y_fault_true must be binary (0 or 1)")
    if not np.all(np.isin(y_sev_true, [0, 1])):
        raise ValueError("y_sev_true must be binary (0 or 1)")
    # Check probabilities in [0,1]
    if not (np.isfinite(y_fault_pred_prob).all() and np.isfinite(y_sev_pred_prob).all()):
        raise ValueError("Predicted probabilities contain non-finite values")
    if not ((0 <= y_fault_pred_prob).all() and (y_fault_pred_prob <= 1).all()):
        raise ValueError("y_fault_pred_prob must be in [0, 1]")
    if not ((0 <= y_sev_pred_prob).all() and (y_sev_pred_prob <= 1).all()):
        raise ValueError("y_sev_pred_prob must be in [0, 1]")
    # Check at most one 1 per fault in severity labels
    for i in range(3):
        if (y_sev_true[:, i*3:(i+1)*3].sum(axis=1) > 1).any():
            raise ValueError(f"Severity true labels for fault {i} have more than one 1 per sample")
    # Check not empty
    if n_samples == 0:
        raise ValueError("Input arrays are empty")

    # ------------------ Fault Diagnosis Metrics ------------------
    # Threshold predicted probabilities at 0.5
    y_fault_pred = (y_fault_pred_prob >= 0.5).astype(int)
    # EMR_F: all 3 faults correct
    EMR_F = float(np.mean(np.all(y_fault_true == y_fault_pred, axis=1)))
    # Macro-F1_F: average F1 across 3 faults
    macro_F1_F = float(f1_score(y_fault_true, y_fault_pred, average='macro', zero_division=0))
    # Recall, precision, FAR per fault
    recall_f = []
    precision_f = []
    far_f = []
    for i in range(3):
        y_true = y_fault_true[:, i]
        y_pred = y_fault_pred[:, i]
        recall = recall_score(y_true, y_pred, zero_division=0)
        precision = precision_score(y_true, y_pred, zero_division=0)
        # FAR = FP / (FP + TN)
        fp = np.sum((y_pred == 1) & (y_true == 0))
        tn = np.sum((y_pred == 0) & (y_true == 0))
        far = fp / (fp + tn) if (fp + tn) > 0 else 0.0
        recall_f.append(round(recall, 4))
        precision_f.append(round(precision, 4))
        far_f.append(round(far, 4))

    # ------------------ Severity Diagnosis Metrics ------------------
    # For each fault, select argmax of predicted probabilities (one-hot)
    y_sev_pred = np.zeros_like(y_sev_pred_prob, dtype=int)
    for i in range(3):
        idx = np.argmax(y_sev_pred_prob[:, i*3:(i+1)*3], axis=1)
        y_sev_pred[np.arange(n_samples), i*3 + idx] = 1
    # EMR_S: all 9 severity labels correct (per sample)
    EMR_S = float(np.mean(np.all(y_sev_true == y_sev_pred, axis=1)))
    # Macro-F1_S: average F1 across 9 classes (flattened)
    macro_F1_S = float(f1_score(y_sev_true, y_sev_pred, average='macro', zero_division=0))
    # Per-severity-level (SLT, MED, SEV) metrics: average across faults
    sev_names = ['SLT', 'MED', 'SEV']
    recall_s = {k: [] for k in sev_names}
    precision_s = {k: [] for k in sev_names}
    far_s = {k: [] for k in sev_names}
    for j, name in enumerate(sev_names):
        for i in range(3):
            y_true = y_sev_true[:, i*3 + j]
            y_pred = y_sev_pred[:, i*3 + j]
            recall = recall_score(y_true, y_pred, zero_division=0)
            precision = precision_score(y_true, y_pred, zero_division=0)
            fp = np.sum((y_pred == 1) & (y_true == 0))
            tn = np.sum((y_pred == 0) & (y_true == 0))
            far = fp / (fp + tn) if (fp + tn) > 0 else 0.0
            recall_s[name].append(recall)
            precision_s[name].append(precision)
            far_s[name].append(far)
    # Average across faults, round to 4 decimals
    recall_s = {k: round(np.mean(v), 4) for k, v in recall_s.items()}
    precision_s = {k: round(np.mean(v), 4) for k, v in precision_s.items()}
    far_s = {k: round(np.mean(v), 4) for k, v in far_s.items()}

    # ------------------ Output Dictionary ------------------
    results = {
        'EMR_F': round(EMR_F, 4),
        'EMR_S': round(EMR_S, 4),
        'macro-F1_F': round(macro_F1_F, 4),
        'macro-F1_S': round(macro_F1_S, 4),
        'fault_recall': recall_f,
        'fault_precision': precision_f,
        'fault_FAR': far_f,
        'severity_recall': recall_s,
        'severity_precision': precision_s,
        'severity_FAR': far_s
    }
    return results

# ------------------ Sample Usage Example ------------------
if __name__ == "__main__":
    np.random.seed(42)
    n = 5
    # True fault labels: (n, 3)
    y_fault_true = np.array([
        [1, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [0, 0, 1],
        [1, 0, 1]
    ])
    # Predicted fault probabilities: (n, 3)
    y_fault_pred_prob = np.array([
        [0.8, 0.2, 0.1],
        [0.3, 0.7, 0.2],
        [0.6, 0.6, 0.4],
        [0.1, 0.2, 0.9],
        [0.7, 0.3, 0.8]
    ])
    # True severity labels: (n, 9) (3 faults × 3 levels)
    y_sev_true = np.zeros((n, 9), dtype=int)
    y_sev_true[0, 0] = 1  # ERP slight
    y_sev_true[1, 4] = 1  # ED medium
    y_sev_true[2, 2] = 1  # ERP severe
    y_sev_true[2, 3] = 1  # ED slight
    y_sev_true[3, 8] = 1  # EU severe
    y_sev_true[4, 1] = 1  # ERP medium
    y_sev_true[4, 8] = 1  # EU severe
    # Predicted severity probabilities: (n, 9)
    y_sev_pred_prob = np.random.rand(n, 9)
    # Evaluate
    results = evaluate_diagnosis_performance(
        y_fault_true, y_fault_pred_prob,
        y_sev_true, y_sev_pred_prob
    )
    print("Evaluation Results:")
    for k, v in results.items():
        print(f"{k}: {v}")

Evaluation Results:
EMR_F: 1.0
EMR_S: 0.0
macro-F1_F: 1.0
macro-F1_S: 0.0556
fault_recall: [1.0, 1.0, 1.0]
fault_precision: [1.0, 1.0, 1.0]
fault_FAR: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
severity_recall: {'SLT': np.float64(0.0), 'MED': np.float64(0.0), 'SEV': np.float64(0.3333)}
severity_precision: {'SLT': np.float64(0.0), 'MED': np.float64(0.0), 'SEV': np.float64(0.1111)}
severity_FAR: {'SLT': np.float64(0.25), 'MED': np.float64(0.4333), 'SEV': np.float64(0.4111)}


In [12]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm

def generate_synthetic_kpi_data(
    scenario: str,
    n_samples: int,
    fault_condition: list,
    severity_levels: dict
) -> pd.DataFrame:
    """
    Generate synthetic KPI data for SON fault diagnosis (CQI, RSRP, SINR, TP)
    for UMa, UMi, RMa scenarios, with configurable faults and severities.

    Parameters:
        scenario: 'UMa', 'UMi', or 'RMa'
        n_samples: number of samples to generate
        fault_condition: list of faults (subset of {'ERP', 'ED', 'EU'})
        severity_levels: dict mapping faults to severity ('slight', 'medium', 'severe')

    Returns:
        pd.DataFrame with columns ['CQI', 'RSRP', 'SINR', 'TP']
    """

    # --- Scenario Configs (Table I) ---
    scenario = scenario.upper()
    if scenario not in ['UMA', 'UMI', 'RMA']:
        raise ValueError("Scenario must be one of 'UMa', 'UMi', 'RMa' (case-insensitive).")
    configs = {
        'UMA': {'bs_power': 49, 'roi': 500, 'inter_bs': 500, 'freq': 2.0, 'bw': 20, 'ue_speed': 3},
        'UMI': {'bs_power': 44, 'roi': 200, 'inter_bs': 200, 'freq': 2.6, 'bw': 20, 'ue_speed': 3},
        'RMA': {'bs_power': 49, 'roi': 1732, 'inter_bs': 1732, 'freq': 0.8, 'bw': 20, 'ue_speed': 3}
    }
    cfg = configs[scenario]

    # --- Fault Parameter Effects (Table II) ---
    # ERP: power loss (dB)
    ERP_loss = {'slight': 10, 'medium': 20, 'severe': 30}
    # ED: downtilt (deg)
    ED_tilt = {'slight': 10, 'medium': 15, 'severe': 20}
    # EU: uptilt (deg)
    EU_tilt = {'slight': -2, 'medium': -3, 'severe': -7}

    # --- Baseline KPI Distributions (Normal) ---
    # CQI: mean 11, std 2, truncated [1, 15]
    cqi_mean, cqi_std = 11, 2
    # RSRP: mean -95, std 8, truncated [-144, -44]
    rsrp_mean, rsrp_std = -95, 8
    # SINR: mean 15, std 4, truncated [0, 30]
    sinr_mean, sinr_std = 15, 4
    # TP: mean 10, std 3, truncated [0, 30]
    tp_mean, tp_std = 10, 3

    # --- Fault Effects ---
    # ERP: reduces CQI, RSRP, SINR, TP
    if 'ERP' in fault_condition:
        sev = severity_levels.get('ERP', 'slight')
        loss = ERP_loss[sev]
        cqi_mean -= {10:2, 20:4, 30:6}[loss]
        rsrp_mean -= loss
        sinr_mean -= loss // 2
        tp_mean -= loss // 3
    # ED: downtilt reduces RSRP, SINR, TP
    if 'ED' in fault_condition:
        sev = severity_levels.get('ED', 'slight')
        tilt = ED_tilt[sev]
        rsrp_mean -= tilt // 2
        sinr_mean -= tilt // 3
        tp_mean -= tilt // 4
    # EU: uptilt increases SINR, TP slightly, but may reduce RSRP
    if 'EU' in fault_condition:
        sev = severity_levels.get('EU', 'slight')
        tilt = EU_tilt[sev]
        rsrp_mean += tilt // 2  # negative tilt reduces mean
        sinr_mean += abs(tilt) // 2
        tp_mean += abs(tilt) // 3

    # --- Truncated Normal Sampling Helper ---
    def tnorm(mean, std, low, high, size):
        a, b = (low - mean) / std, (high - mean) / std
        return truncnorm.rvs(a, b, loc=mean, scale=std, size=size)

    # --- Generate KPIs ---
    CQI = np.round(tnorm(cqi_mean, cqi_std, 1, 15, n_samples)).astype(int)
    RSRP = tnorm(rsrp_mean, rsrp_std, -144, -44, n_samples)
    SINR = tnorm(sinr_mean, sinr_std, 0, 30, n_samples)
    TP = tnorm(tp_mean, tp_std, 0, 30, n_samples)

    # --- Clip to valid ranges (edge cases) ---
    CQI = np.clip(CQI, 1, 15)
    RSRP = np.clip(RSRP, -144, -44)
    SINR = np.clip(SINR, 0, 30)
    TP = np.clip(TP, 0, 30)

    # --- Return DataFrame ---
    df = pd.DataFrame({
        'CQI': CQI,
        'RSRP': RSRP,
        'SINR': SINR,
        'TP': TP
    })
    return df

# ------------------ Sample Usage Example ------------------
if __name__ == "__main__":
    # Example: UMa, 10 samples, ERP (severe) and ED (medium)
    scenario = 'UMa'
    n_samples = 10
    fault_condition = ['ERP', 'ED']
    severity_levels = {'ERP': 'severe', 'ED': 'medium'}
    df = generate_synthetic_kpi_data(scenario, n_samples, fault_condition, severity_levels)
    df.head()

In [13]:
df.head()

,CQI,RSRP,SINR,TP
0,6,-120.364146,1.645700,2.150841
1,4,-140.312502,0.344330,2.386420
2,5,-137.403549,3.142195,0.149366
3,5,-141.854354,0.177371,0.814376
4,3,-134.646287,6.968343,0.237294


In [14]:
import itertools
import pandas as pd

# --- Use the generate_synthetic_kpi_data function from previous cell ---

def generate_full_synthetic_dataset(n_per_case=200):
    scenarios = ['UMa', 'UMi', 'RMa']
    faults = ['ERP', 'ED', 'EU']
    severities = ['slight', 'medium', 'severe']
    all_data = []

    # Normal (no fault) for each scenario
    for scenario in scenarios:
        df = generate_synthetic_kpi_data(
            scenario=scenario,
            n_samples=n_per_case,
            fault_condition=[],
            severity_levels={}
        )
        df['Scenario'] = scenario
        df['Faults'] = 'Normal'
        df['Severities'] = ''
        all_data.append(df)

    # All single and multi-fault combinations
    for scenario in scenarios:
        for n_faults in range(1, 4):
            for fault_combo in itertools.combinations(faults, n_faults):
                # For each fault, try all severity combinations
                for severity_combo in itertools.product(severities, repeat=n_faults):
                    fault_condition = list(fault_combo)
                    severity_levels = {f: s for f, s in zip(fault_combo, severity_combo)}
                    df = generate_synthetic_kpi_data(
                        scenario=scenario,
                        n_samples=n_per_case,
                        fault_condition=fault_condition,
                        severity_levels=severity_levels
                    )
                    df['Scenario'] = scenario
                    df['Faults'] = '+'.join(fault_condition)
                    df['Severities'] = '+'.join([f"{f}:{s}" for f, s in severity_levels.items()])
                    all_data.append(df)

    # Concatenate all
    full_df = pd.concat(all_data, ignore_index=True)
    return full_df

# Example: Generate and save the full dataset
full_dataset = generate_full_synthetic_dataset(n_per_case=200)
print(full_dataset.head())
print("Total samples:", len(full_dataset))
# Optionally save to CSV
full_dataset.to_csv('synthetic_kpi_full_dataset.csv', index=False)

   CQI        RSRP       SINR         TP Scenario  Faults Severities
0   10 -109.398789  20.151082  10.429746      UMa  Normal           
1   12  -95.690721   8.240535  10.542849      UMa  Normal           
2   12  -94.143211  12.680529   8.254042      UMa  Normal           
3   13  -99.508140  21.592319  12.212510      UMa  Normal           
4   11  -93.162488  19.910258   7.337363      UMa  Normal           
Total samples: 38400
